# Imports

In [195]:
import torch
set_seed = torch.manual_seed(0)

# Data generator

In [196]:
def regression_data_generator(n, k=2):
    '''
    n is the number of samples per minibatch
    k is the dimensionality of each vector x
    '''

    x = torch.rand(n, k) #x is nxk
    w = torch.tensor([10.0, -6.0])       #parameter initialization

    y = torch.tensor([torch.dot(w, v_i) for v_i in x])

    return x, y

# Model


In [197]:
class Regression_model:
    def __init__(self, k = 2, sigma = 0.01):
        self.W = torch.normal(mean=0,
                              std=sigma,
                              size=(k,),
                              requires_grad=True)
        self.b = torch.zeros(size=(1,), requires_grad=True)
        
    def forward(self, x):
        return x@(self.W.reshape((-1, 1))) + self.b

# Loss

In [198]:
def l2_loss(y, y_hat):
    '''
    Remember that y (true value) and y_hat (prediction) are nxk
    '''
    
    l = (y_hat-torch.reshape(y, y_hat.shape)) ** 2 / 2
    return l.mean()

# Optimizer

In [199]:
class SGD:
    def __init__(self, parameters, lr = 0.01):
        self.lr = lr
        self.parameters = parameters

    def step(self):
        for p in self.parameters:
            p -= self.lr * p.grad

    def zero_grad(self):
        for p in self.parameters:
            if p.grad is not None:
                p.grad.zero_()

# Trainer

In [204]:
class Trainer:
    def __init__(self, model, loss, optimizer, iterations = 20):
        self.model = model
        self.loss = loss
        self.optimizer = optimizer
        self.iterations = iterations

    def fit_epoch(self, data_generator):
        loss = 0
        x, y = data_generator(self.iterations)
        y_hat = self.model.forward(x)
        loss += self.loss(y, y_hat)
        self.optimizer.zero_grad()
        with torch.no_grad():
            loss.backward()
            self.optimizer.step()

        return loss

# Training tests


In [205]:
model = Regression_model()
optimizer = SGD([model.W, model.b])
trainer = Trainer(model, l2_loss, optimizer)


loss = []
for i in range(5):      #5 epochs
    loss.append(trainer.fit_epoch(regression_data_generator))
print(loss)

[tensor(4.6078, grad_fn=<AddBackward0>), tensor(8.3818, grad_fn=<AddBackward0>), tensor(5.9978, grad_fn=<AddBackward0>), tensor(8.2849, grad_fn=<AddBackward0>), tensor(6.4236, grad_fn=<AddBackward0>)]
